# Análise dos dados da API de deputados

Análise dos dados obtidos por meio da API da Câmara com dados de deputados



In [1]:
# Importando as bibliotecas
import pandas as pd
import requests

# URL de acesso aos dados via api
url = 'https://dadosabertos.camara.leg.br/api/v2/deputados?itens=100'

# Pandas consegue ler JSON diretamente
# df = pd.read_json(url)
# ISSO NUNCA FUNCIONA
# Vamos ter que requisitar o arquivo, transformar em lista, pegar apenas o elemento desejado e transformar em DF

# Obtendo os dados
resposta = requests.get(url)

# Criando o DataFrame
df = pd.DataFrame(resposta.json()['dados'])

df.head()

,id,idLegislatura,nome,siglaPartido,siglaUf,uri,uriPartido,urlFoto
0,178957,55,ABEL MESQUITA JR.,DEM,RR,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
1,178864,55,ADAIL CARNEIRO,PODE,CE,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
2,178914,55,ADALBERTO CAVALCANTI,AVANTE,PE,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
3,178890,55,ADELMO CARNEIRO LEÃO,PT,MG,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
4,178968,55,ADELSON BARRETO,PR,SE,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...


In [2]:
df.count()

id               100
idLegislatura    100
nome             100
siglaPartido     100
siglaUf          100
uri              100
uriPartido       100
urlFoto          100
dtype: int64

Por padrão, a API retorna apenas 100 deputados por vez.

Então precisamos fazer novas requisições para obter os demais deputados

In [3]:
# Como obter os links para as próximas páginas
resposta.json()['links'][1]['href']

'https://dadosabertos.camara.leg.br/api/v2/deputados?&pagina=2&itens=100'

In [4]:
# Separar os links da resposta
for item in resposta.json()['links']:    
    if item['rel'] == 'self':
        atual = item['href']
    elif item['rel'] == 'last':
        ultimo = item['href']
    elif item['rel'] == 'next':
        proximo = item['href']

# Enquanto a pagina atual for diferente da ultima pagina
while atual != ultimo:
    # Requisita os dados
    resposta = requests.get(proximo)
    # Adiciona no df
    df = df.append(pd.DataFrame(resposta.json()['dados']))
    
    # Separar os links da resposta
    for item in resposta.json()['links']:
        if item['rel'] == 'self':
            atual = item['href']
        elif item['rel'] == 'last':
            ultimo = item['href']
        elif item['rel'] == 'next':
            proximo = item['href']
        
df.count()

id               512
idLegislatura    512
nome             512
siglaPartido     512
siglaUf          512
uri              512
uriPartido       512
urlFoto          512
dtype: int64

In [5]:
df.head()

,id,idLegislatura,nome,siglaPartido,siglaUf,uri,uriPartido,urlFoto
0,178957,55,ABEL MESQUITA JR.,DEM,RR,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
1,178864,55,ADAIL CARNEIRO,PODE,CE,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
2,178914,55,ADALBERTO CAVALCANTI,AVANTE,PE,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
3,178890,55,ADELMO CARNEIRO LEÃO,PT,MG,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...
4,178968,55,ADELSON BARRETO,PR,SE,https://dadosabertos.camara.leg.br/api/v2/depu...,https://dadosabertos.camara.leg.br/api/v2/part...,http://www.camara.leg.br/internet/deputado/ban...


In [6]:
# Quantos partidos?
df.groupby('siglaUf').siglaPartido.value_counts()

siglaUf  siglaPartido
AC       PT               3
         MDB              2
         DEM              1
         PSB              1
         PSDB             1
AL       AVANTE           1
         PDT              1
         PHS              1
         PP               1
         PR               1
         PSB              1
         PSD              1
         PSDB             1
         PT               1
AM       PSDB             2
         DEM              1
         MDB              1
         PDT              1
         PP               1
         PR               1
         PRB              1
AP       MDB              1
         PCdoB            1
         PODE             1
         PP               1
         PR               1
         PSB              1
         PSD              1
BA       PT               7
         PP               5
                         ..
SE       PRB              1
         PSB              1
         PSC              1
         PSD              

In [7]:
# Aprendendo a usar query
# Podemos filtrar o DataFrame de acordo com algum critério
# No caso vamos pegar apenas os deputados do DEM e contar a quantidade por UF
df.query('siglaPartido == "DEM"').siglaUf.value_counts()

RJ    8
SP    5
BA    4
MG    4
TO    2
PE    2
MS    2
AC    1
SC    1
MT    1
PB    1
PA    1
DF    1
RN    1
PI    1
RS    1
CE    1
PR    1
RR    1
AM    1
RO    1
MA    1
ES    1
Name: siglaUf, dtype: int64

In [8]:
# Usando query para restringir a busca no DataFrame
# Agora vamos filtrar os deputados do DEM que são de São Paulo, e vamos mostrar seus nomes
df.query('siglaPartido == "DEM" and siglaUf == "SP"').nome

17             ALEXANDRE LEITE
44            ELI CORRÊA FILHO
42         JORGE TADEU MUDALEN
52    MISSIONÁRIO JOSÉ OLIMPIO
21              RODRIGO GARCIA
Name: nome, dtype: object